# Imports

In [6]:
import time
import threading
import multiprocessing

# Sequential Execution

In [3]:
# Function to calculate the sum from 1 to n
def sequential_sum(n):
    return sum(range(1, n+1))

# Input for large number
n = 10000000

# Measure execution time
start_time = time.time()
sum_result = sequential_sum(n)
end_time = time.time()

execution_time = end_time - start_time

print(f"Sequential Sum: {sum_result}")
print(f"Execution Time: {execution_time} seconds")



Sequential Sum: 50000005000000
Execution Time: 0.13345861434936523 seconds


# Threads execution

In [4]:
# Function to calculate the sum of a specific range
def thread_sum(start, end, result, index):
    result[index] = sum(range(start, end + 1))

# Function to calculate the sum in parallel using threading
def parallel_sum_threaded(n, num_threads):
    threads = []
    result = [0] * num_threads
    range_size = n // num_threads

    # Create threads for different parts of the range
    for i in range(num_threads):
        start = i * range_size + 1
        end = (i + 1) * range_size if i != num_threads - 1 else n
        thread = threading.Thread(target=thread_sum, args=(start, end, result, i))
        threads.append(thread)
        thread.start()

    # Join all threads
    for thread in threads:
        thread.join()

    return sum(result)

# Measure execution time for threaded approach
num_threads = 4
start_time = time.time()
sum_result_threaded = parallel_sum_threaded(n, num_threads)
end_time = time.time()

execution_time_threaded = end_time - start_time

print(f"Threaded Sum: {sum_result_threaded}")
print(f"Execution Time (Threaded): {execution_time_threaded} seconds")


Threaded Sum: 50000005000000
Execution Time (Threaded): 0.10765528678894043 seconds


# Trials With processes

In [7]:
# Function to calculate the sum of a specific range (for multiprocessing)
def process_sum(start, end):
    return sum(range(start, end + 1))

# Function to calculate the sum in parallel using multiprocessing
def parallel_sum_multiprocessing(n, num_processes):
    pool = multiprocessing.Pool(processes=num_processes)
    range_size = n // num_processes
    ranges = [(i * range_size + 1, (i + 1) * range_size if i != num_processes - 1 else n) for i in range(num_processes)]

    # Compute the sums in parallel
    results = pool.starmap(process_sum, ranges)

    return sum(results)

# Measure execution time for multiprocessing approach
num_processes = 4
start_time = time.time()
sum_result_multiprocessing = parallel_sum_multiprocessing(n, num_processes)
end_time = time.time()

execution_time_multiprocessing = end_time - start_time

print(f"Multiprocessing Sum: {sum_result_multiprocessing}")
print(f"Execution Time (Multiprocessing): {execution_time_multiprocessing} seconds")


Multiprocessing Sum: 50000005000000
Execution Time (Multiprocessing): 0.05491828918457031 seconds


# Questions

#### How does the execution time change when moving from sequential to threaded to multiprocessing implementations?
- Sequential: Execution time is generally the longest, as it is a single-threaded, single-process computation.
- Threaded: Execution time can be reduced, especially if the task is I/O-bound or if there are multiple cores available. Threading can improve performance by overlapping I/O-bound tasks.
- Multiprocessing: Execution time can be significantly reduced compared to sequential and threaded approaches, especially for CPU-bound tasks. Multiprocessing runs in parallel on multiple CPU cores, which can lead to significant speedup.

#### Calculating speedup

In [9]:
speedup_thread = execution_time/execution_time_threaded
speedup_processes = execution_time/execution_time_multiprocessing

print(f"The speedup using threads is {speedup_thread}")
print(f"The speedup using processes is {speedup_processes}")

The speedup using threads is 1.239684722692835
The speedup using processes is 2.430130587295523


#### Computing the Efficiency 

In [10]:
np = 4
efficiency_thread = speedup_thread/np
efficiency_processes = speedup_processes/np

print(f"The efficiency using threads is {efficiency_thread}")
print(f"The efficiency using processes is {efficiency_processes}")

The efficiency using threads is 0.30992118067320873
The efficiency using processes is 0.6075326468238808


#### Amdhal and Gustafson Laws

In [11]:
P = 0.70
amdhal_s=1/((1-P) + (P/np))
print(amdhal_s)

2.1052631578947367


In [12]:
alpha = 1- P
gustafson= np + alpha*(1-np)
print(gustafson)

3.0999999999999996


### Are there any performance differences between the threaded and multiprocessing versions?
Yes, multiprocessing is generally better for CPU-bound tasks, as it avoids Global Interpreter Lock (GIL) limitations in Python, while threading is more suitable for I/O-bound tasks. Threading can provide speedup, but it may be limited by the GIL when working with CPU-intensive operations.

### What challenges did you face when implementing parallelism, and how did you address them?
- Threading GIL: Python’s GIL prevents true parallel execution in threads for CPU-bound tasks. I handled this by using multiprocessing for CPU-heavy tasks.
- Task Division: Dividing the range of numbers efficiently is important for balanced workload across threads or processes. I ensured each thread or process worked on a roughly equal portion of the range.

### When would you choose threading over multiprocessing or vice versa for parallel tasks?
- Threading: Useful for I/O-bound tasks like reading from files or making network requests.
- Multiprocessing: More suitable for CPU-bound tasks, as it avoids the GIL and utilizes multiple cores effectively.